In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import random
import scipy

from PIL import Image
from os import getcwd

from tqdm import tqdm
from mpl_toolkits.axes_grid1 import make_axes_locatable

%matplotlib inline

input_dir = getcwd() + '/img/input_samples/'

In [ ]:
###########################################################################################################################
#                                                      Array utils                                                        #
###########################################################################################################################


def normalize(arr, zero_padding=False, offset_coef=0.001):
    min_val = np.min(arr)
    max_val = np.max(arr)
    offset = offset_coef * abs(min_val)
    new_arr = arr - min_val if zero_padding else arr - min_val + offset
    d = (max_val - min_val) if zero_padding else (max_val - min_val + offset)
    new_arr = new_arr / d
    return new_arr


def normalize_img(arr):
    arr_min = np.min(arr)
    arr_max = np.max(arr)
    if arr_min > 0 and arr_max < 255: 
        return arr
    else:
        arr_norm = 255 * (arr - arr_min) / (arr_max - arr_min)
        return arr_norm


def threshold_arr_1d(arr, th_val, use_abs=False):
    th_val = abs(th_val) if use_abs else th_val
    for i in range(len(arr)):
        if arr[i] < th_val:
            arr[i] = th_val
    return arr
    
    
def threshold_arr_2d(arr, th_val, use_abs=False):
    for i in range(arr.shape[0]):
        arr[i, :] = threshold_arr_1d(arr[i, :], th_val, use_abs)
    return arr


###########################################################################################################################
#                                                       FFT utils                                                         #
###########################################################################################################################


def find_ft_1d(img):
    ft = np.fft.fft(img)
    return np.fft.fftshift(ft)


def find_ift_1d(ft):
    ift = np.fft.ifftshift(ft)
    return np.fft.ifft(ift)


def find_ft_2d(img):
    ft = np.fft.fft2(img)
    return np.fft.fftshift(ft)


def find_ift_2d(ft):
    ift = np.fft.ifftshift(ft)
    return np.fft.ifft2(ift)


def freq_numbers_1d(size):
    if size % 2:
        return np.arange(-(size // 2), size // 2 + 1, 1) 
    else:
        return np.arange(-(size // 2), size // 2, 1) 
    

def freq_arr_1d(size, step=1):
    freq = freq_numbers_1d(size) 
    return freq / step / size


def freq_numbers_2d(size):
    x_size, y_size = size
    x_freq_numbers = freq_numbers_1d(x_size)
    y_freq_numbers = freq_numbers_1d(y_size)
    return x_freq_numbers, y_freq_numbers
    

def freq_arr_2d(size, x_step=1, y_step=1):
    x_size, y_size = size
    x_freq_numbers, y_freq_numbers = freq_numbers_2d(size) 
    x_freq = x_freq_numbers / x_step / x_size
    y_freq = y_freq_numbers / y_step / y_size
    return x_freq, y_freq


###########################################################################################################################
#                                                   Frequency domain filters                                              #
###########################################################################################################################


def freq_filter_2d(x_freq, y_freq):
    x, y = np.meshgrid(x_freq, y_freq)
    f = np.hypot(x, y)
    return f


# should be deprecated or changed
"""
def freq_filter(x_freq, y_freq, factor=2.4):
    eps = 10 ** -8
    x, y = np.meshgrid(x_freq, y_freq)
    f = np.hypot(x, y)
    f = f ** factor + eps
    return normalize(1 / f)
"""


def freq_pink_filter_2d(x_freq, y_freq, factor=1, zero_padding=False, no_mean=False):
    x, y = np.meshgrid(x_freq, y_freq)
    f = np.hypot(x, y)
    f = 1 / (1 + np.abs(f))
    f = f ** factor
    f = np.where(f==1, 0, f) if no_mean else f
    return normalize(f, zero_padding)


def freq_pink_filter_1d(x_freq, factor=1, no_mean=False):
    f = 1 / (1 + np.abs(x_freq))
    f = f ** factor
    f = np.where(f==1, 0, f) if no_mean else f
    return normalize(f, zero_padding)


# replaced by no_mean flag in freq_filter_1d
"""
def freq_filter_1d_a(x_freq, factor=1):
    f = 1 / np.where(x_freq == 0, 1, np.abs(x_freq))
    f = f ** factor
    f[len(x_freq) // 2] = 0
    return f
"""


def freq_sharp_round_filter_2d(x_freq, y_freq, radius, low_pass_filter=True):
    f = np.ones((np.size(x_freq), np.size(y_freq)))
    check = gt if low_pass_filter else lt
    rr = radius ** 2

    for i, x in enumerate(x_freq):
        for j, y in enumerate(y_freq):
            if check(x ** 2 + y ** 2, rr):
                f[i, j] = 0
    return f


def freq_sharp_square_filter(x_freq, y_freq, width, angle=0, low_pass_filter=True):
    f = np.ones((np.size(x_freq), np.size(y_freq)))
    check = np.greater if low_pass_filter else np.less
    angle_radians = math.radians(angle)
    rotation_matrix = np.array([[math.cos(angle_radians), -math.sin(angle_radians)],
                                [math.sin(angle_radians), math.cos(angle_radians)]])
    
    for i, x in enumerate(x_freq):
        for j, y in enumerate(y_freq):
            rotated_x, rotated_y = np.dot(rotation_matrix, np.array([x, y])) 

            if check(abs(rotated_x) + abs(rotated_y), width):
                f[i, j] = 0
    return f


###########################################################################################################################
#                                                Spatial domain utils (NOT REWORKED)                                      #
###########################################################################################################################


def spatial_smooth_filter(x_size, y_size, depth, horiz=True):
    values = np.linspace(0, 1, depth)
    values = 6 * values ** 5 - 15 * values ** 4 + 10 * values ** 3
    values = 1 - values
    if horiz:
        kernel = np.tile(values, (y_size, 1))
    else:
        kernel = values[:, np.newaxis] * np.ones((1, x_size))   
    return kernel


def make_img_transition_x(img, depth, is_dx_pos=True):
    y_size, x_size = img.shape
    additional_img = gen_cloud(x_size + depth, y_size)   
    transition_kernel = spatial_smooth_filter(x_size, y_size, depth)     
    
    new_img = np.copy(img)
    if is_dx_pos:
        new_img[:, -depth:x_size] = img[:, -depth:x_size] * transition_kernel + \
                                additional_img[:, 0:depth] * (1 - transition_kernel)
        return new_img, additional_img[:, depth:]    
    else:
        transition_kernel = np.fliplr(transition_kernel)
        new_img[:, 0:depth] = img[:, 0:depth] * transition_kernel + \
                          additional_img[:, -depth:] * (1 - transition_kernel)  
        return new_img, additional_img[:, 0:-depth]    


def make_img_transition_y(img, depth, is_dy_pos=True):
    y_size, x_size = img.shape
    additional_img = gen_cloud(x_size, y_size + depth)   
    transition_kernel = spatial_smooth_filter(x_size, y_size, depth, horiz=False)
        
    new_img = np.copy(img)
    if is_dy_pos:
        new_img[-depth:x_size, :] = img[-depth:x_size, :] * transition_kernel + \
                                additional_img[0:depth, :] * (1 - transition_kernel)
        return new_img, additional_img[depth:, :]    
    else:
        transition_kernel = np.flipud(transition_kernel)
        new_img[0:depth, :] = img[0:depth, :] * transition_kernel + \
                          additional_img[-depth:, :] * (1 - transition_kernel)  
        return new_img, additional_img[0:-depth:1, :]    



###########################################################################################################################
#                                                       Other (NOT REWORKED)                                              #
###########################################################################################################################


def fit_clement(x_freq, y_freq, alpha1, alpha2, eta=0.1, angle=3):
    x, y = np.meshgrid(x_freq, y_freq)
    f = np.hypot(x, y )
    fp = abs(angle * x  + y)
    f = np.sqrt((1 - eta) * f ** 2 + eta * fp ** 2)    
    f = (1 + abs(f)) ** (-alpha1) + 0.02 * (1 + abs(f)) ** (-alpha2)
    return f


def surrogates(x, ns, tol_pc=5., verbose=True, maxiter=1E6, sorttype="quicksort"):
    # as per the steps given in Lancaster et al., Phys. Rep (2018)
    nx = x.shape[0]
    xs = np.zeros((ns, nx))
    maxiter = 10000
    ii = np.arange(nx)

    # get the fft of the original array
    x_amp = np.abs(np.fft.fft(x))
    x_srt = np.sort(x)
    r_orig = np.argsort(x)

    # loop over surrogate number
    pb_fmt = "{desc:<5.5}{percentage:3.0f}%|{bar:30}{r_bar}"
    pb_desc = "Estimating IAAFT surrogates ..."
    for k in tqdm(range(ns), bar_format=pb_fmt, desc=pb_desc,
                  disable=not verbose):

        # 1) Generate random shuffle of the data
        count = 0
        r_prev = np.random.permutation(ii)
        r_curr = r_orig
        z_n = x[r_prev]
        percent_unequal = 100.

        # core iterative loop
        while (percent_unequal > tol_pc) and (count < maxiter):
            r_prev = r_curr

            # 2) FFT current iteration yk, and then invert it but while
            # replacing the amplitudes with the original amplitudes but
            # keeping the angles from the FFT-ed version of the random
            y_prev = z_n
            fft_prev = np.fft.fft(y_prev)
            phi_prev = np.angle(fft_prev)
            e_i_phi = np.exp(phi_prev * 1j)
            z_n = np.fft.ifft(x_amp * e_i_phi)

            # 3) rescale zk to the original distribution of x
            r_curr = np.argsort(z_n, kind=sorttype)
            z_n[r_curr] = x_srt.copy()
            percent_unequal = ((r_curr != r_prev).sum() * 100.) / nx

            # 4) repeat until number of unequal entries between r_curr and 
            # r_prev is less than tol_pc percent
            count += 1

        if count >= (maxiter - 1):
            print("maximum number of iterations reached!")

        xs[k] = np.real(z_n)

    return xs


def adjust_freq_1d(img):
    return np.append(img, abs(img[0]))


def adjust_img_1d(img):
    return np.append(img, img[0])
        
        
def gen_cloud(x_size, y_size, factor=2.4):
    xx = np.linspace(-x_size / 2, x_size / 2, x_size)
    yy = np.linspace(-y_size / 2, y_size / 2, y_size)
    whitenoise = np.random.normal(0, 1, (y_size, x_size))
    cloud_freq = find_ft_2d(whitenoise)  
    kernel = freq_filter(xx, yy, factor=factor)
    cloud_freq_filtered = cloud_freq * kernel
    cloud_spatial = find_ift_2d(cloud_freq_filtered).real
    return normalize_img(cloud_spatial)


# Reworked
def show_images(*images, vmin=0, vmax=255, x_fig_size=10, cmap='gray', y_fig_size=10, graphs_per_row=2):
    row_num = len(images) // graphs_per_row + 1 if len(images) % graphs_per_row else len(images) // graphs_per_row
    col_num = len(images) // row_num + 1 if len(images) % row_num else len(images) // row_num
    
    f, axes = plt.subplots(row_num, col_num, figsize=(x_fig_size, y_fig_size))
    if row_num == 1 and col_num == 1:
        axes = [axes]
    
    for ax, img in zip(axes.flatten(), images):
        im = ax.imshow(img, cmap=cmap, vmin=vmin, vmax=vmax, aspect='equal')  # Set aspect to 'equal'
        divider = make_axes_locatable(ax)
        cax = divider.append_axes("right", size="5%", pad=0.05)
        plt.colorbar(im, cax=cax)

    plt.tight_layout()
    plt.show()


def lin_regression(x, y):
    # y - original img
    # x - restored img
    num = np.mean(x * y) - np.mean(x) * np.mean(y)
    denum = np.mean(x ** 2) - np.mean(x) ** 2
    a = num / denum
    b = np.mean(y) - a * np.mean(x)
    return a, b


def lin_phase(start, end, size):
    pos_freq = np.linspace(start, end, size // 2)
    neg_freq = -pos_freq[::-1]
    return np.append(neg_freq, pos_freq)

### Cumulus test

In [ ]:
img = Image.open(input_dir + '2.jpg').convert('L')
img -= np.mean(img)
img_fr = find_ft_2d(img)

x_size, y_size = img_fr.shape
xx = freq_numbers_1d(x_size)
yy = freq_numbers_1d(y_size)

magn = np.abs(img_fr)
magn = normalize(magn)
phase = np.angle(img_fr)

In [ ]:
fig = plt.figure(figsize=(10, 10))

ax1 = fig.add_subplot(2, 2, 1)
ax1.imshow(img, cmap='gray')

ax2 = fig.add_subplot(2, 2, 2)
ax2.imshow(np.log(magn), cmap='gray')

ax3 = fig.add_subplot(2, 2, 3)
ax3.imshow(phase, cmap='gray')

ax4 = fig.add_subplot(2, 2, 4)
ax4.imshow(find_ift_2d(magn * np.exp(1j * phase)).real, cmap='gray')

In [ ]:
phase_fr = find_ft_2d(phase)
phase_magn = np.abs(phase_fr)
phase_angle = np.angle(phase_fr)

fig = plt.figure(figsize=(10, 10))

ax1 = fig.add_subplot(1, 2, 1)
ax1.imshow(phase_magn, cmap='gray')

ax2 = fig.add_subplot(1, 2, 2)
ax2.imshow(phase_angle, cmap='gray')

In [ ]:
ph_fr = np.sqrt(normalize(phase_magn)) * np.exp(1j * phase_angle)
ph = find_ift_2d(ph_fr).real

plt.imshow(ph, cmap='gray')

In [ ]:
fig = go.Figure()
fig.add_trace(go.Surface(x=xx, y=yy, z=np.log10(magn)))

fig.update_layout(
    scene = dict(
    zaxis = dict(range=[-5, 0],),
))

fig.show()

In [ ]:
fig = go.Figure()
# fig.add_trace(go.Surface(x=list(range(20)), y=list(range(20)), z=phase[x_size // 2 - 10: x_size // 2 + 10, y_size // 2 - 10: y_size // 2 + 10]))
fig.add_trace(go.Surface(x=xx, y=yy, z=phase))

In [ ]:
# Phase along x axis
# plt.plot(phase[x_size // 2 + 2, y_size // 2:])

# Phase along y axis
plt.plot(phase[x_size // 2:, y_size // 2 ])

### Generating pseudo random phase trying to mimic same phase distribution along each axis

In [ ]:
new_phase_y = np.zeros((x_size, y_size))

for i in range(x_size):
    new_phase_y[i, :] = surrogates(phase[i, :], 1)
    
    
new_phase_x = np.zeros((x_size, y_size))

for i in range(x_size):
    new_phase_x[:, i] = surrogates(phase[:, i], 1)

new_phase_sur = new_phase_x + new_phase_y
plt.imshow(new_phase_sur, cmap='gray')

In [ ]:
# Restoring image using original magnitude and new phase 

restored_img_fr = magn * np.exp(1j * new_phase_sur)
restored_img = find_ift_2d(restored_img_fr).real

plt.imshow(restored_img, cmap='gray')

In [ ]:
# Restoring image using 1/f kernel for magnitude and new phase 
# Results are equal to regular FFT syntesis

kernel = freq_pink_filter_2d(xx, yy)
restored_img_fr = kernel * np.exp(1j * new_phase_sur)
restored_img = find_ift_2d(restored_img_fr).real

fig = plt.figure(figsize=(10, 10))

ax1 = fig.add_subplot(2, 2, 1)
ax1.imshow(np.log(kernel), cmap='gray')

ax2 = fig.add_subplot(2, 2, 2)
ax2.imshow(restored_img, cmap='gray')

### Restoring image using linearly changing phase

In [ ]:
initial_phase = [np.pi / 3, 2 * np.pi / 3, np.pi, np.pi / 2, 3 * np.pi / 2]
initial_phase += [-i for i in initial_phase]
whitenoise = np.random.normal(0, 1, (y_size, x_size))
whitenoise_fr = find_ft_2d(whitenoise)
lin_random_phase = np.angle(whitenoise_fr)

width = 5
scale = 0.2
alpha_x = 1.4
alpha_y = 1.
step = 1

for i in range(-width, width + 1, step):
    initial_phase_x = random.choice(initial_phase)
    initial_phase_y = random.choice(initial_phase)
    freq_y = lin_phase(initial_phase_y, initial_phase_y + random.choice(initial_phase), y_size)
    freq_x = lin_phase(initial_phase_x, initial_phase_x + random.choice(initial_phase), x_size)
    lin_random_phase[lin_random_phase.shape[0] // 2 + i, :] = freq_y
    lin_random_phase[:, lin_random_phase.shape[1] // 2 + i] = freq_x

restored_img_fr = magn * np.exp(1j * lin_random_phase)
restored_img = find_ift_2d(restored_img_fr).real

fig = plt.figure(figsize=(10, 10))

ax1 = fig.add_subplot(2, 2, 1)
ax1.imshow(lin_random_phase, cmap='gray')

ax2 = fig.add_subplot(2, 2, 2)
ax2.imshow(restored_img, cmap='gray')

In [ ]:
restored_img_fr = kernel * np.exp(1j * lin_random_phase)
restored_img = find_ift_2d(restored_img_fr).real

fig = plt.figure(figsize=(10, 10))

ax1 = fig.add_subplot(2, 2, 1)
ax1.imshow(lin_random_phase, cmap='gray')

ax2 = fig.add_subplot(2, 2, 2)
ax2.imshow(restored_img, cmap='gray')

### 1D regression

In [ ]:
# Custom regression formulas

# y_log = np.log(y_vals)
# x_log = np.log(1 + x_vals)

# alpha = -2 * np.sum(y_log * x_log) / np.sum(x_log ** 2) 
# alpha

In [ ]:
# Approximation along Y-axis

alpha = 1.
dy = 1
factor_y = [1, 0.2, 0.2, 0.15, 0.25, 0.08]
const_y = [0.0019, 0.0008, 0.0015, 0.001, 0.0012, 0.0008]

y_vals = magn[x_size // 2 + dy, magn.shape[1] // 2:] 
x_vals = np.array(range(len(y_vals)))

w_y = factor_y[abs(dy)] * ((1 + abs(x_vals)) ** (-alpha) + const_y[abs(dy)])
# kernel = freq_pink_filter_2d(xx, yy, factor=1)
# kernel[magn.shape[0] // 2, :] = w

plt.plot(x_vals, w_y)
plt.plot(x_vals[1:], y_vals[1:])
plt.yscale('log')

In [ ]:
# Approximation along X-axis

alpha = 1.45
dx = 1
factor_x = [1, 1, 1, 1, 1, 1]
const_x = [0.00015, 0.00015, 0.00015, 0.00015, 0.00015, 0.00015]

y_vals = magn[magn.shape[0] // 2:, y_size // 2 + dx] 
x_vals = np.array(range(len(y_vals)))
w_x = factor_x[abs(dx)] * ((1 + abs(x_vals)) ** (-alpha) + const_x[abs(dx)])

plt.plot(x_vals, w_x)
plt.plot(x_vals[0:], y_vals[0:])
plt.yscale('log')

In [ ]:
# Approximation along Y = X

alpha1 = 1.8
alpha2 = 1.0

l = magn.shape[0] // 2
y_vals = np.zeros(l)

for i in range(l):
    y_vals[i] = magn[l + i, l + i]
x_vals = np.array(range(len(y_vals))) * 2 ** 0.5

y_log = np.log(y_vals)
x_log = np.log(1 + x_vals)

w_diag = (1 + abs(x_vals)) ** (-alpha1) + 0.02 * (1 + abs(x_vals)) ** (-alpha2)
# kernel[:, magn.shape[1] // 2] = w

plt.plot(x_vals, w_diag)
plt.plot(x_vals[1:], y_vals[1:])
plt.yscale('log')

In [ ]:
# Combined kernel with previous approximation results

width = 5
scale = 0.2
alpha_x = 1.4
alpha_y = 1.
step = 1

# kernel = freq_pink_filter_2d(xx, yy, factor=0.5)
kernel = fit_clement(xx, yy, alpha1, alpha2)
# kernel = np.zeros((x_size, y_size))

for i in range(-width, width + 1, step):
    w_y = factor_y[abs(i)] * ((1 + abs(yy)) ** (-alpha_y) + const_y[abs(i)])
    w_x = factor_x[abs(i)] * ((1 + abs(xx)) ** (-alpha_x) + const_x[abs(i)])
    kernel[magn.shape[0] // 2 + i, :] = w_y
    kernel[:, magn.shape[1] // 2 + i] = w_x
    
# kernel[magn.shape[0] // 2, :] = w_y 
# kernel[:, magn.shape[1] // 2] = w_x 
kernel = normalize(kernel)

fig = go.Figure()
fig.add_trace(go.Surface(x=xx, y=yy, z=np.log10(kernel)))

'''
fig.update_layout(
    scene = dict(
        zaxis = dict(range=[-5, 0],)
    )
)
'''

fig.show()

In [ ]:
cloud_freq_filtered = kernel * np.exp(1j * phase)
cloud_spatial = find_ift_2d(cloud_freq_filtered).real
a, b = lin_regression(cloud_spatial, img)
cloud_spatial = a * cloud_spatial + b

plt.imshow(cloud_spatial, cmap='gray', vmin=np.min(img), vmax=np.max(img))
# plt.imshow(cloud_spatial, cmap='gray')

In [ ]:
whitenoise = np.random.normal(0, 1, (y_size, x_size))
whitenoise_fr = find_ft_2d(whitenoise)
random_phase = np.angle(whitenoise_fr)

cloud_freq_filtered = kernel * np.exp(1j * random_phase)
cloud_spatial = find_ift_2d(cloud_freq_filtered).real

plt.imshow(cloud_spatial, cmap='gray')

In [ ]:
cloud_freq_filtered = kernel * np.exp(1j * lin_random_phase)
cloud_spatial = find_ift_2d(cloud_freq_filtered).real

plt.imshow(cloud_spatial, cmap='gray')

### 2D regression

In [ ]:
y_vals = magn
x_vals = freq_filter_2d(xx, yy)

y_log = np.log(y_vals)
x_log = np.log(1 + x_vals)

alpha = -2 * np.sum(y_log * x_log) / np.sum(x_log ** 2) 
kernel = freq_pink_filter_2d(xx, yy, factor=alpha / 2)
alpha

In [ ]:
fig = go.Figure()
fig.add_trace(go.Surface(x=xx, y=yy, z=np.log10(kernel)))
fig.add_trace(go.Surface(x=xx, y=yy, z=np.log10(magn)))

In [ ]:
cloud_freq_filtered = kernel * np.exp(1j * phase)
cloud_spatial = find_ift_2d(cloud_freq_filtered).real
a, b = lin_regression(cloud_spatial, img)
cloud_spatial = a * cloud_spatial + b

plt.imshow(cloud_spatial, cmap='gray', vmin=np.min(img), vmax=np.max(img))

In [ ]:
cloud_freq_filtered = kernel * np.exp(1j * random_phase)
cloud_spatial = find_ift_2d(cloud_freq_filtered).real

plt.imshow(cloud_spatial, cmap='gray')

In [ ]:
cloud_freq_filtered = kernel * np.exp(1j * lin_random_phase)
cloud_spatial = find_ift_2d(cloud_freq_filtered).real

plt.imshow(cloud_spatial, cmap='gray')

### New approach

In [ ]:
x_arr, y_arr = np.meshgrid(xx, yy)
window = 1 - normalize(np.sqrt(x_arr ** 2 + y_arr ** 2))

windowed_img = window * img
window_ft = find_ft_2d(window)
windowed_img_ft = find_ft_2d(windowed_img)

# magn_windowed = np.abs(windowed_img_ft) - np.abs(window_ft)
# angle_windowed = np.angle(window_ft) - np.angle(windowed_img_ft)
    
magn_windowed = np.abs(windowed_img_ft - window_ft)
phase_windowed = np.angle(windowed_img_ft - window_ft)
restored_img = find_ift_2d(magn_windowed * np.exp(1j * phase_windowed)).real

vmin = np.min(windowed_img)
vmax = np.max(windowed_img)


plt.imshow(windowed_img, cmap='gray')
plt.colorbar()
plt.show()

plt.imshow(np.log(magn_windowed), cmap='gray')
plt.colorbar()
plt.show()

plt.imshow(phase_windowed, cmap='gray')
plt.colorbar()
plt.show()

plt.imshow(restored_img, cmap='gray')
plt.colorbar()
plt.show()

In [ ]:
windowed_img_ft = find_ft_2d(phase_windowed)
m = np.abs(windowed_img_ft)
a = np.angle(windowed_img_ft)

whitenoise = np.random.normal(0, 1, (x_size, y_size))
psd = m * m
restored_phase = find_ift_2d(psd ** 0.5 * np.abs(whitenoise) * np.exp(1j * a)).real

kernel = freq_pink_filter_2d(xx, yy, factor=0.5)
cloud_hf = find_ift_2d(magn_windowed * np.exp(1j * restored_phase)).real
a, b = lin_regression(cloud_hf, img)
cloud_hf = a * cloud_hf + b

plt.imshow(np.log(m), cmap='gray')
plt.colorbar()
plt.show()

plt.imshow(restored_phase, cmap='gray')
plt.colorbar()
plt.show()

plt.imshow(cloud_hf, cmap='gray', vmin=vmin, vmax=vmax)
plt.colorbar()
plt.show()

In [ ]:
alpha1 = 1.8
alpha2 = 1.0
alpha_x = 1.4
alpha_y = 1.

whitenoise = np.random.normal(0, 1, (x_size, y_size))
whitenoise_fr = find_ft_2d(whitenoise)
kernel = fit_clement(xx, yy, alpha1, alpha2, eta=0.5)
cloud_lf = find_ift_2d(whitenoise_fr * kernel).real
a, b = lin_regression(cloud_lf, img)
cloud_lf = a * cloud_lf + b

plt.imshow(np.log(kernel), cmap='gray')
plt.colorbar()

In [ ]:
plt.imshow(cloud_hf + 0.001 * cloud_lf, cmap='gray')
plt.colorbar()